In [ ]:
# %load_ext tensorboard

import sys
import os

import config

import Run_qlattice

import re
import hashlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import NN_run
import prepare_data as data_source
import mongodb_driver
import requests
import feyn
import pickle
import NN_utility as NN_u

import utility_func as uf
import sklearn.metrics as skm 
from matplotlib import pyplot as plt
import numpy.ma as ma
from sklearn.model_selection import train_test_split

import rfet_model as rmdl

import matplotlib.patches as patches


str_run         = "run_regular_grid_common_goal"
str_electrode   = "lat2"

min_step = 0.02
# min_steps = [ 0.05,0.02,0.01,0.001,0.0001,0.00001,0.0]
# min_steps = [ 0.02]

In [ ]:
# mdb = mongodb_driver.mdb_connect("mdb_cmodel", "felixl", "mdbpass")
mdb = mongodb_driver.mdb_connect("mdb_cmodel", "root", "t(jWbtU4DRnuXqdhK")

mdb_col = mdb["NN"]


# Q Model (symbolic regression)

In [ ]:
pars = {
    "data_type"                         : 'DC',
    "data_path"                         : f"{config.dir_data}/{str_run}_{str_electrode}_minstep{min_step}_export.pkl",
    "save_path"                         : f"Qlattice/models/mx",

    "mdl_type"                          : 'Q',
    "mdl_arch"                          : 'SR',
    "augmentation_type"                 : None,

    "sr_num_epochs"                     : 2,
    "sr_max_complexity"                 : 50,
    "sr_train_size"                     : 0.765,
    "sr_transfer"                       : None,
    # "sr_transfer"                       : "Thung",

    
    "user"                              : os.getlogin(),
    "comment"                           : "minimal example test"
}



In [ ]:
_id_q = mdb_col.insert_one(pars)
_id_q = str(_id_q.inserted_id)
print(_id_q)

Q_list = ["Qbg", "Qfg", "Qtg", "Qlat2", "Qlat1"]

data = data_source.load_dc_ramp_data(pars['data_path'], Xlabels =["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"], Ylabel=Q_list)

# select BG = lat1 = 0V
data = data[data["Vbglat1"] == 0]

Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]

res = {}

for Q in Q_list:
    
    Ylabel  =   [Q]
    (train, test, best_model, mae_test_real, mse_test_real, r2_test_real ) = Run_qlattice.mx_run_ql(random_seed=0,
                                num_epochs= pars["sr_num_epochs"],
                                bound_ilat_to= None,
                                sigma_gaus_aug= pars["augmentation_type"],
                                max_complexity= pars["sr_max_complexity"],
                                train_size = pars["sr_train_size"],
                                transfer=pars["sr_transfer"],
                                save_path=pars["save_path"],
                                # starting_model=[feyn.Model.load('./Qlattice/models/Set2_transfer1.json')],
                                starting_model=None,
                                scale = True,
                                data = data,
                                Xlabels = Xlabels,
                                Ylabel = Ylabel,
                                sorting_crit='bic',
                                id = _id_q + "_" + Q ,
                                # id = "qfg_test",
                                mdl_type = pars["mdl_type"],
                                scaling= 1E15,
                                splitting_algorithm="kennard_stone"
                                )
    
    res[Q] = { "test" : test, "model" : best_model, "mae_test" : mae_test_real, "mse_test" : mse_test_real, "r2_test": r2_test_real }


In [ ]:
print(_id_q)
